### TEST GUI

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import csv
from PyQt5.QtWidgets import (QApplication, QMainWindow, QPushButton, QFileDialog, 
                             QVBoxLayout, QWidget, QLineEdit, QLabel, QComboBox)
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from matplotlib import cm

class ImageGUI(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("2D NumPy Array Viewer")
        
        self.image = None
        self.points = []
        self.colormap = "viridis"
        self.is_interaction_enabled = True  # Flag to track interaction state
        
        self.initUI()
    
    def initUI(self):
        widget = QWidget()
        layout = QVBoxLayout()
        
        self.canvas = FigureCanvas(Figure())
        self.ax = self.canvas.figure.add_subplot(111)
        self.toolbar = NavigationToolbar(self.canvas, self)
        layout.addWidget(self.toolbar)  # Add zoom and pan functionality
        layout.addWidget(self.canvas)
        
        self.load_button = QPushButton("Load Image")
        self.load_button.clicked.connect(self.load_image)
        layout.addWidget(self.load_button)
        
        self.clear_button = QPushButton("Clear Picks")
        self.clear_button.clicked.connect(self.clear_picks)
        layout.addWidget(self.clear_button)
        
        self.vmin_label = QLabel("vmin:")
        layout.addWidget(self.vmin_label)
        self.vmin_entry = QLineEdit("0")
        layout.addWidget(self.vmin_entry)
        
        self.vmax_label = QLabel("vmax:")
        layout.addWidget(self.vmax_label)
        self.vmax_entry = QLineEdit("255")
        layout.addWidget(self.vmax_entry)
        
        self.cmap_label = QLabel("Colormap:")
        layout.addWidget(self.cmap_label)
        self.cmap_dropdown = QComboBox()
        self.cmap_dropdown.addItems(sorted(cm.cmap_d.keys()))
        self.cmap_dropdown.currentTextChanged.connect(self.update_colormap)
        layout.addWidget(self.cmap_dropdown)
        
        self.update_button = QPushButton("Update Colormap")
        self.update_button.clicked.connect(self.update_colormap)
        layout.addWidget(self.update_button)
        
        self.export_button = QPushButton("Export Points to CSV")
        self.export_button.clicked.connect(self.export_to_csv)
        layout.addWidget(self.export_button)
        
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        
        self.canvas.mpl_connect('button_press_event', self.on_click)
        self.toolbar.actionTriggered.connect(self.toggle_interaction)  # Listen for toolbar actions
    
    def toggle_interaction(self, action):
        if action.text() in ["Pan", "Zoom"]:
            self.is_interaction_enabled = False
        else:
            self.is_interaction_enabled = True
    
    def load_image(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "Open File", "", "NumPy files (*.npy)")
        if file_path:
            self.image = np.load(file_path)
            self.vmin, self.vmax = np.min(self.image), np.max(self.image)
            self.display_image()
    
    def display_image(self):
        self.ax.clear()
        cmap = self.cmap_dropdown.currentText()
        self.ax.imshow(self.image, cmap=cmap, vmin=float(self.vmin_entry.text()), vmax=float(self.vmax_entry.text()))
        self.ax.set_title("Click to Connect Points")
        self.canvas.draw()
    
    def update_colormap(self):
        if self.image is not None:
            self.display_image()
    
    def on_click(self, event):
        if event.inaxes and self.is_interaction_enabled:
            x, y = int(event.xdata), int(event.ydata)
            if len(self.points) > 0:
                last_x, last_y = self.points[-1]
                interpolated_points = self.interpolate_points(last_x, last_y, x, y)
                self.points.extend(interpolated_points)
            self.points.append((x, y))
            self.update_points()
    
    def interpolate_points(self, x1, y1, x2, y2):
        num_points = max(abs(x2 - x1), abs(y2 - y1)) + 1
        x_values = np.linspace(x1, x2, num_points, dtype=int)
        y_values = np.linspace(y1, y2, num_points, dtype=int)
        return list(zip(x_values, y_values))
    
    def update_points(self):
        self.ax.clear()
        cmap = self.cmap_dropdown.currentText()
        self.ax.imshow(self.image, cmap=cmap, vmin=float(self.vmin_entry.text()), vmax=float(self.vmax_entry.text()))
        if len(self.points) > 1:
            x_vals, y_vals = zip(*self.points)
            self.ax.plot(x_vals, y_vals, 'ro-', markersize=5)
        self.canvas.draw()
    
    def clear_picks(self):
        self.points = []
        self.update_points()
    
    def export_to_csv(self):
        file_path, _ = QFileDialog.getSaveFileName(self, "Save File", "", "CSV files (*.csv)")
        if file_path:
            with open(file_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["X", "Y"])
                writer.writerows(self.points)

if __name__ == "__main__":
    # Example usage:
    # Save a random 2D NumPy array as an .npy file for testing
    test_array = np.random.rand(100, 100) * 255
    np.save("test_image.npy", test_array)
    
    app = QApplication(sys.argv)
    window = ImageGUI()
    window.show()
    sys.exit(app.exec_())


AttributeError: module 'matplotlib.cm' has no attribute 'cmap_d'